### Modifying PASCAL VOC labels

This notebook shows how to split the images into a grid and modify the existing labels. This is needed when using pre-trained CNN's which are trained on images of size relatively smaller than those in the dataset. 

While detecting smaller objects, resizing the large images into a small input size will make these smaller objects loose their features. Hence we divide the large images into 2 by 2 or 3 by 3 grid and generate new annotation files with modified coordinates. 

note: Overlapping annotations between two partitions are logged.


In [ ]:
import numpy as np 
import matplotlib as mp
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import misc
import sys
from xml.etree import ElementTree
from xml.etree.ElementTree import Element, SubElement
from lxml import etree
import os

In [ ]:

XML_EXT = '.xml'
ENCODE_METHOD = 'utf-8'

class PascalVocReader:

    def __init__(self, filepath, q_y, q_x, h, w):
        # shapes type:
        # [labbel, [(x1,y1), (x2,y2), (x3,y3), (x4,y4)], color, color, difficult]
        self.shapes = []
        self.filepath = filepath
        self.verified = False
        self.q_x = q_x
        self.q_y = q_y
        self.h = h
        self.w = w
        self.x_origin = q_x * w
        self.y_origin = q_y * h
        self.overlap = False
        print ("origin"+str(self.x_origin) + "," + str(self.y_origin))

    def parseXML(self):
        assert self.filepath.endswith(XML_EXT), "Unsupport file format"
        parser = etree.XMLParser(encoding=ENCODE_METHOD)
        tree = ElementTree.parse(self.filepath, parser=parser)
        xmltree = tree.getroot()
        filename = xmltree.find('filename')
        old_filename = filename.text
        filename.text = old_filename.replace(".jpg", "_"+str(self.q_x)+"_"+str(self.q_y)+".jpg") #new filename
        
        path = xmltree.find('path')
        old_path = path.text
        path.text = old_path.replace(".jpg", "_"+str(self.q_x)+"_"+str(self.q_y)+".jpg")
        
        size = xmltree.find('size')
        width = size.find('width')
        height = size.find('height')
        width.text = str(self.w)
        height.text = str(self.h)
               

        for object_iter in xmltree.findall('object'):
            bndbox = object_iter.find("bndbox")
            label = object_iter.find('name').text
            
            xmin = int(bndbox.find('xmin').text) 
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            print (str(xmin) +","+str(xmax)+","+str(ymin)+","+str(ymax))
            if xmin >= self.x_origin and xmax <= (self.x_origin + self.w) and ymin >= self.y_origin and ymax <= (self.y_origin + self.h):
                bndbox.find('xmin').text = str(xmin - self.x_origin)
                bndbox.find('xmax').text = str(xmax - self.x_origin)
                bndbox.find('ymin').text = str(ymin - self.y_origin)
                bndbox.find('ymax').text = str(ymax - self.y_origin)
                print ("bdbox accepted n"+str(self.q_x) + str(self.q_y))
            else:
                print ("bnbox removed in"+str(self.q_x) + str(self.q_y))
                self.overlap = True
                xmltree.remove(object_iter)
        if len(xmltree.findall('object')) != 0:
            print("executed")
            new_filepath = self.filepath.replace(".xml", "_"+str(self.q_x)+"_"+str(self.q_y)+".xml")
            tree.write(new_filepath)
            return True
        else:
            return False



def modify_labeldata(path,gridSize=3):
    #set the directory paths
    image_dir = os.path.join(path, "testing")
    anno_dir = os.path.join(path, "test_annotations")
    
    H=0
    W=0
    c=0
    H_out = 0
    W_out = 0
    anno_list = os.listdir(anno_dir)
    #list the images and annotations
    for i,file in enumerate(os.listdir(image_dir)):
        if(".jpg" in file):
            img = misc.imread(os.path.join(image_dir,file))
            if i==0:
                H, W,c = img.shape
                H_out = H//gridSize
                W_out = W//gridSize
            overlap = False
            for j in range(gridSize):  # rows
                for k in range(gridSize): #columns
                    x_origin = j * W_out
                    y_origin = k * H_out
                    img_crop = img[x_origin : x_origin + H_out , y_origin : y_origin + W_out]
                    # read the .xml and get the objects in it
                    anno_name = file.replace(".jpg", ".xml")
                    if anno_name in anno_list:
                        # for each object
                        reader = PascalVocReader(os.path.join(anno_dir,anno_name), j, k, H_out,W_out)
                        res = reader.parseXML()
                        if reader.overlap:
                            overlap = True
                        if res :
                            new_imgpath = file.replace(".jpg", "_"+str(k)+"_"+str(j)+".jpg")
                            misc.imsave(os.path.join(image_dir,new_imgpath),img_crop)
            if overlap:
                if not os.path.exists(os.path.join(path,"log")):
                    os.makedirs(os.path.join(path,"log"))
                os.rename(os.path.join(image_dir,file),os.path.join(path,"log",file))


modify_labeldata("/home/pushyamik/Desktop/labeldata")